In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [7]:
plt.rcParams.update({'figure.autolayout': True})

In [8]:
# ---------- CONFIG ----------
MOVIES_CSV = "movies.csv"
USERS_CSV = "users.csv"
HISTORY_CSV = "watch_history.csv"
OUTPUT_DIR = Path("analysis_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

In [9]:
# Binge parameters
BINGE_WINDOW_HOURS = 24    # episodes within this many hours considered part of same binge
BINGE_MIN_EPISODES = 2     # at least these many episodes in a session => binge
TOP_N_GENRES = 10

In [10]:
def safe_read_csv(path):
    try:
        return pd.read_csv(path)
    except FileNotFoundError:
        raise SystemExit(f"Missing file: {path} - please place it in the working folder.")
    except Exception as e:
        raise SystemExit(f"Error reading {path}: {e}")


In [11]:
# Load data
movies = safe_read_csv(MOVIES_CSV)
users = safe_read_csv(USERS_CSV)
history = safe_read_csv(HISTORY_CSV)

In [12]:
# Basic validation (warn if expected columns missing)
expected_movies = {'movie_id','title','content_type','genre_primary','duration_minutes','imdb_rating','is_netflix_original','added_to_platform'}
expected_users = {'user_id','email','first_name','last_name','subscription_plan','created_at'}
expected_history = {'session_id','user_id','movie_id','watch_date','device_type','watch_duration_minutes','progress_percentage','action','quality','location_country','is_download','user_rating'}

missing_movies = expected_movies - set(movies.columns)
missing_users = expected_users - set(users.columns)
missing_history = expected_history - set(history.columns)

if missing_movies:
    print("Warning - movies.csv missing columns (analysis will still attempt to continue):", missing_movies)
if missing_users:
    print("Warning - users.csv missing columns (analysis will still attempt to continue):", missing_users)
if missing_history:
    print("Warning - watch_history.csv missing columns (analysis will still attempt to continue):", missing_history)

In [13]:
# ---- CLEAN & NORMALIZE ----
# Trim whitespace in column names and values
movies.columns = movies.columns.str.strip()
users.columns = users.columns.str.strip()
history.columns = history.columns.str.strip()

In [14]:
# Standardize types
history['watch_date'] = pd.to_datetime(history['watch_date'], errors='coerce')
history['watch_duration_minutes'] = pd.to_numeric(history.get('watch_duration_minutes', 0), errors='coerce').fillna(0)
# progress as numeric percent (0-100)
history['progress_percentage'] = pd.to_numeric(history.get('progress_percentage', np.nan), errors='coerce')
history['user_rating'] = pd.to_numeric(history.get('user_rating', np.nan), errors='coerce')
movies['imdb_rating'] = pd.to_numeric(movies.get('imdb_rating', np.nan), errors='coerce')
movies['duration_minutes'] = pd.to_numeric(movies.get('duration_minutes', np.nan), errors='coerce')

In [15]:
# Fill missing genre
movies['genre_primary'] = movies.get('genre_primary', pd.Series()).fillna('Unknown')

In [16]:
# Enrich history with movie metadata
events = history.merge(
    movies[['movie_id','title','content_type','genre_primary','number_of_seasons','number_of_episodes','duration_minutes','is_netflix_original','imdb_rating']],
    on='movie_id', how='left'
)

In [17]:
# compute watch hours
events['watch_hours'] = events['watch_duration_minutes'] / 60.0

In [18]:
# ---- ANALYSIS 1: Top Genres ----
genre_agg = events.groupby('genre_primary').agg(
    total_watch_hours=('watch_hours','sum'),
    unique_viewers=('user_id', lambda s: s.nunique()),
    avg_user_rating=('user_rating', lambda s: pd.to_numeric(s, errors='coerce').mean()),
    avg_imdb_rating=('imdb_rating','mean'),
    title_count=('movie_id', lambda s: s.nunique())
).sort_values('total_watch_hours', ascending=False)

genre_agg.to_csv(OUTPUT_DIR / "genre_aggregation.csv")
print("Top genres by watch hours:\n", genre_agg.head(TOP_N_GENRES))

Top genres by watch hours:
                total_watch_hours  unique_viewers  avg_user_rating  \
genre_primary                                                       
Adventure            7388.013333            4914         3.307081   
War                  6254.690000            4349         3.326104   
Animation            6220.280000            4461         3.333870   
Sci-Fi               6075.090000            4286         3.408567   
Action               6004.261667            4353         3.335135   
Comedy               5966.028333            4418         3.336907   
Romance              5737.123333            3988         3.417667   
History              5621.306667            4071         3.371580   
Western              5485.930000            4208         3.296780   
Biography            5396.508333            4016         3.385042   

               avg_imdb_rating  title_count  
genre_primary                                
Adventure             6.426647           68  
War  

In [19]:
# Plot: top genres by watch hours
top_genres = genre_agg.head(TOP_N_GENRES).reset_index()
plt.figure(figsize=(10,6))
plt.bar(top_genres['genre_primary'], top_genres['total_watch_hours'])
plt.xticks(rotation=45, ha='right')
plt.title("Top Genres by Total Watch Hours")
plt.ylabel("Total Watch Hours")
plt.savefig(OUTPUT_DIR / "top_genres_watch_hours.png")
plt.close()

In [20]:
# ---- ANALYSIS 2: Watch Hours per User ----
user_agg = events.groupby('user_id').agg(
    total_watch_hours=('watch_hours','sum'),
    avg_session_minutes=('watch_duration_minutes','mean'),
    titles_watched=('movie_id','nunique'),
    sessions_count=('session_id','nunique')
).sort_values('total_watch_hours', ascending=False)

user_agg.to_csv(OUTPUT_DIR / "user_watch_hours.csv")
print("\nTop users by watch hours (sample):\n", user_agg.head(10))


Top users by watch hours (sample):
             total_watch_hours  avg_session_minutes  titles_watched  \
user_id                                                              
user_07566          49.681667           229.300000               9   
user_02222          44.926667           192.542857              13   
user_07872          43.875000           239.318182              10   
user_06269          41.568333           166.273333              14   
user_03869          40.558333           143.147059              17   
user_00979          40.071667           120.215000              15   
user_00811          39.310000           138.741176              16   
user_03046          39.120000           130.400000              14   
user_08254          38.868333           166.578571              12   
user_00161          38.670000           145.012500              14   

            sessions_count  
user_id                     
user_07566               9  
user_02222              13  
user_0

In [21]:
# Histogram of watch hours per user (cap at 99th percentile for readability)
cap = user_agg['total_watch_hours'].quantile(0.99)
plt.figure(figsize=(8,5))
plt.hist(user_agg['total_watch_hours'].clip(upper=cap), bins=40)
plt.title("Distribution of Total Watch Hours per User (capped at 99th percentile)")
plt.xlabel("Total Watch Hours")
plt.ylabel("Number of Users")
plt.savefig(OUTPUT_DIR / "watch_hours_per_user_hist.png")
plt.close()

In [22]:
# ---- ANALYSIS 3: Binge Behavior ----
# We'll define sessions and binge:
# Approach A (session-level): keep chronological events per user; group contiguous watches of same title within BINGE_WINDOW_HOURS.
df = events.copy().sort_values(['user_id','title','watch_date'])
# time since previous watch of same title by same user
df['prev_watch_dt'] = df.groupby(['user_id','title'])['watch_date'].shift(1)
df['hours_since_prev'] = (df['watch_date'] - df['prev_watch_dt']).dt.total_seconds() / 3600.0
df['new_session'] = (df['hours_since_prev'].isna()) | (df['hours_since_prev'] > BINGE_WINDOW_HOURS)
df['session_group'] = df.groupby(['user_id','title'])['new_session'].cumsum()  # integer group id per (user,title)
# identify sessions with count >= BINGE_MIN_EPISODES
session_stats = df.groupby(['user_id','title','session_group']).agg(
    episodes_watched=('session_id','count'),
    session_start=('watch_date','min'),
    session_end=('watch_date','max'),
    session_hours=('watch_hours','sum')
).reset_index()
session_stats['is_binge'] = session_stats['episodes_watched'] >= BINGE_MIN_EPISODES

In [23]:
# user-level binge summary
user_binge = session_stats.groupby('user_id').agg(
    total_sessions=('is_binge','count'),
    binge_sessions=('is_binge','sum'),
    binge_rate=('is_binge', lambda s: s.sum()/s.count() if s.count() else 0),
    avg_episodes_per_session=('episodes_watched','mean')
).sort_values('binge_sessions', ascending=False)

user_binge.to_csv(OUTPUT_DIR / "user_binge_summary.csv")
session_stats.to_csv(OUTPUT_DIR / "session_stats.csv")
print("\nUser binge sample:\n", user_binge.head(10))


User binge sample:
             total_sessions  binge_sessions  binge_rate  \
user_id                                                  
user_02416              21               6    0.285714   
user_09616              16               5    0.312500   
user_02935              18               5    0.277778   
user_05421              15               5    0.333333   
user_08051              11               5    0.454545   
user_01668              12               5    0.416667   
user_05192              15               5    0.333333   
user_01099              16               5    0.312500   
user_00979              15               5    0.333333   
user_06290              13               5    0.384615   

            avg_episodes_per_session  
user_id                               
user_02416                  1.285714  
user_09616                  1.562500  
user_02935                  1.277778  
user_05421                  1.333333  
user_08051                  1.454545  
user_0166

In [24]:
# Plot: distribution of episodes per session
plt.figure(figsize=(8,5))
plt.hist(session_stats['episodes_watched'].clip(upper=10), bins=10)
plt.title("Distribution of Episodes per Session (capped at 10)")
plt.xlabel("Episodes in Session")
plt.ylabel("Number of Sessions")
plt.savefig(OUTPUT_DIR / "episodes_per_session_hist.png")
plt.close()

In [25]:
# ---- ANALYSIS 4: Ratings vs Genres ----
# Use user_rating where available; fallback to imdb_rating.
events['effective_rating'] = events['user_rating'].fillna(events['imdb_rating'])
rating_by_genre = events.dropna(subset=['effective_rating']).groupby('genre_primary')['effective_rating'].agg(['mean','median','count']).sort_values('mean', ascending=False)
rating_by_genre.to_csv(OUTPUT_DIR / "rating_by_genre.csv")
print("\nRatings by genre (sample):\n", rating_by_genre.head(10))


Ratings by genre (sample):
                    mean  median  count
genre_primary                         
Drama          5.907751     6.5   4090
Fantasy        5.832451     6.0   4382
Thriller       5.770649     5.5   3390
Adventure      5.762673     6.2   7070
Western        5.732510     5.7   5223
Crime          5.714919     6.0   4672
Horror         5.691212     5.9   4768
Sci-Fi         5.672937     6.2   5247
Animation      5.655200     5.8   5846
History        5.583504     6.0   5262


In [26]:
# boxplot for top genres (by event count)
top_genres_for_box = events['genre_primary'].value_counts().head(8).index.tolist()
vals = [events.loc[events['genre_primary']==g, 'effective_rating'].dropna().values for g in top_genres_for_box]
plt.figure(figsize=(12,6))
plt.boxplot(vals, labels=top_genres_for_box, showfliers=False)
plt.title("Ratings distribution by genre (top genres by events)")
plt.ylabel("Rating")
plt.xticks(rotation=45)
plt.savefig(OUTPUT_DIR / "ratings_by_genre_boxplot.png")
plt.close()

C:\Users\janan\AppData\Local\Temp\ipykernel_18072\1796031370.py:5: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(vals, labels=top_genres_for_box, showfliers=False)


In [27]:
# ---- ANALYSIS 5: Top Titles (hit-driven vs long tail) ----
title_stats = events.groupby(['movie_id','title']).agg(
    total_watch_hours=('watch_hours','sum'),
    unique_viewers=('user_id','nunique'),
    avg_progress=('progress_percentage', lambda s: pd.to_numeric(s, errors='coerce').mean())
).sort_values('total_watch_hours', ascending=False).reset_index()
title_stats.head(50).to_csv(OUTPUT_DIR / "top_titles_by_watch_hours.csv", index=False)

print("\nTop titles sample:\n", title_stats.head(10)[['title','total_watch_hours','unique_viewers','avg_progress']])


Top titles sample:
               title  total_watch_hours  unique_viewers  avg_progress
0       Love Battle         271.510000             123     52.221101
1       Mystery Day         249.510000              91     49.864894
2    Secret Mission         240.213333              97     48.614737
3         Dark Love         239.426667             100     50.642157
4      Big Princess         238.630000             102     54.609278
5  Adventure Legend         238.140000             100     46.336170
6        Night Love         234.196667             117     44.399115
7       Hero Empire         233.603333             107     47.127103
8      Storm Family         224.543333             119     50.040708
9       Secret City         223.053333             122     52.301613


In [28]:
# ---- SEGMENTATION HINTS (quick) ----
# e.g., top genres per subscription plan
events_users = events.merge(users[['user_id','subscription_plan','country']], on='user_id', how='left')
genre_by_plan = events_users.groupby(['subscription_plan','genre_primary']).agg(total_watch_hours=('watch_hours','sum')).reset_index()
genre_by_plan.to_csv(OUTPUT_DIR / "genre_by_subscription_plan.csv")

In [29]:
# ---- QUICK INSIGHTS SUMMARY (text) ----
insights = []
# Top genres
topg = genre_agg.head(5)
insights.append("Top genres by total watch hours (top 5):")
for g, row in topg.iterrows():
    insights.append(f"- {g}: {row['total_watch_hours']:.1f} hours, {int(row['unique_viewers'])} unique viewers, avg_imdb={row['avg_imdb_rating']:.2f}")
# Binge
b_rate = user_binge['binge_rate'].median() if not user_binge.empty else 0
insights.append(f"\nMedian user binge_rate (fraction of sessions that are binge-like): {b_rate:.3f}")
# Ratings vs genres
top_rating_genres = rating_by_genre.head(5)
insights.append("\nGenres with highest average effective rating (top 5):")
for g, row in top_rating_genres.iterrows():
    insights.append(f"- {g}: mean={row['mean']:.2f} (n={int(row['count'])})")

with open(OUTPUT_DIR / "insights_summary.txt", "w", encoding='utf-8') as f:
    f.write("\n".join(insights))

In [30]:
# ---- DONE ----
print("\nAll done. Outputs saved to:", OUTPUT_DIR.resolve())
print("Key files: genre_aggregation.csv, user_watch_hours.csv, user_binge_summary.csv, rating_by_genre.csv, top_titles_by_watch_hours.csv, insights_summary.txt")


All done. Outputs saved to: C:\Users\janan\OneDrive\Netflix User Behaviour Analysis\analysis_outputs
Key files: genre_aggregation.csv, user_watch_hours.csv, user_binge_summary.csv, rating_by_genre.csv, top_titles_by_watch_hours.csv, insights_summary.txt
